In [1]:
# -*- coding: UTF-8 -*-
# 环境配置： Python + Splinter 实现浏览器自动化操作入门指南：
# https://zhuanlan.zhihu.com/p/30208753
from splinter import Browser
import splinter

import bibtexparser

In [40]:
browser = Browser('chrome')   #打开谷歌浏览器

In [3]:
GS_root = 'https://scholar.google.com/'

test_url = 'https://scholar.google.com.hk/scholar?q=paper&hl=zh-CN&as_sdt=0,5'
# 一般第一次需要手动人机验证
# todo: 自动绕过谷歌人机验证，可以参考这个链接：
# https://github.com/cassieeric/python_crawler/blob/master/Google_verify/Google_verify.py  
browser.visit(GS_root)        

In [77]:
# 在这里更换要查询的论文名字
browser.visit(GS_root)
time.sleep(0.5)
paper_name = 'Understanding Adversarial Attacks on Deep Learning Based Medical Image Analysis Systems'

# 访问谷歌学术并查找论文
browser.fill('q', paper_name)
button = browser.find_by_xpath('//button[@type="submit"]').click()

In [78]:
# 获取论文信息
browser.find_by_xpath('//a[@title="引用"]').first.click()
# 打开bibtex页面 并获取信息
browser.find_by_xpath('//a[@class="gs_citi"]').click()
bibtex = browser.find_by_xpath('//pre').value
citation_entry = bibtexparser.loads(bibtex).entries[0]

if 'author' in citation_entry.keys():
    names = citation_entry['author']
    names = names.split('and')
    new_names = []
    for name in names:
        new_name = ' '.join([x.replace(' ', '') for x in reversed(name.split(','))])
        new_names.append(new_name)
    citation_entry['author'] = ', '.join(new_names)

print('原本论文信息：')
for key in sorted(citation_entry.keys()):
    print('{:<10}| {:<}'.format(key, citation_entry[key]))
print('#'*30)

# 回退两次返回搜索界面
browser.back()
browser.back()

原本论文信息：
ENTRYTYPE | article
ID        | cheng2020coarse
author    | Yihua Cheng, Shiyao Huang, Fei Wang, Chen Qian, Feng Lu
journal   | arXiv preprint arXiv:2001.00187
title     | A Coarse-to-Fine Adaptive Network for Appearance-Based Gaze Estimation
year      | 2020
##############################


In [79]:
# 找到第一篇论文作为检索到的论文，打开引用链接
cited_links = browser.links.find_by_partial_href('/scholar?cites=')
link = cited_links[0].outer_html.split('="')[1].split('">')[0]
search_link = GS_root + link[1:]
browser.visit(search_link)

In [80]:

# 如果失败或者需要翻页，取消注释并手动设置url
# search_link = 'https://scholar.google.com/scholar?start=10&hl=zh-CN&as_sdt=0,5&sciodt=0,5&cites=14976651643158220927&scipsc='

# 遍历列表，整理引用我们论文的信息
import time
links = browser.find_by_xpath('//a[@title="引用"]')
n_links = len(links)
for ii in range(n_links):
    browser.visit(search_link)
    links = browser.find_by_xpath('//a[@title="引用"]')
    try:
        time.sleep(0.5)
        link = links[ii]
        link.click()
        # 打开bibtex页面 并获取信息
        browser.find_by_xpath('//a[@class="gs_citi"]').click()
        bibtex = browser.find_by_xpath('//pre').value
        citation_entry = bibtexparser.loads(bibtex).entries[0]

        if 'author' in citation_entry.keys():
            names = citation_entry['author']
            names = names.split('and')
            new_names = []
            for name in names:
                new_name = ' '.join([x.replace(' ', '') for x in reversed(name.split(','))])
                new_names.append(new_name)
            citation_entry['author'] = ', '.join(new_names)
        
        print(f'<{ii}> 引用的论文信息：')
        for key in sorted(citation_entry.keys()):
            print('{:<10}| {:<}'.format(key, citation_entry[key]))
        p_name = citation_entry['journal'] if 'journal' in citation_entry.keys() else citation_entry['booktitle'] if 'booktitle' in citation_entry.keys() else ' '
        lazy_copy = '{}\t{}\t\t\t{}'.format(p_name, citation_entry['year'], citation_entry['author'])
        print('Lazy copy: ', lazy_copy)

        print('-'*30)
    
        browser.back()
        browser.back()
    except Exception as e:
        print(f'In {ii}, find error:', e)

#  现在，点击下一页，然后把下一页的链接url赋值给 `search_link`
print('现在请点击下一页')

<0> 引用的论文信息：
ENTRYTYPE | article
ID        | wang2020vr
author    | Miao Wang, Xu-Quan Lyu, Yi-Jun Li, Fang-Lue Zhang
journal   | Computational Visual Media
pages     | 1--26
publisher | Springer
title     | VR content creation and exploration with deep learning: A survey
year      | 2020
Lazy copy:  Computational Visual Media	2020			Miao Wang, Xu-Quan Lyu, Yi-Jun Li, Fang-Lue Zhang
------------------------------
现在请点击下一页


# 以下的代码被弃用，暂时不要管它

In [16]:
with open('summary.csv', 'w') as fid_write:
    is_csv_header = True
    for paper_name in paper_list:
        print(f'Processing paper: {paper_name}')

        # 访问谷歌学术并查找论文
        browser.visit(GS_root)
        browser.fill('q', paper_name)
        browser.find_by_xpath('//button[@type="submit"]').click()

        # 获取论文信息
        browser.find_by_xpath('//a[@title="引用"]').first.click()
        # 打开bibtex页面 并获取信息
        browser.find_by_xpath('//a[@class="gs_citi"]').click()
        bibtex = browser.find_by_xpath('//pre').value
        citation_entry = bibtexparser.loads(bibtex).entries[0]

        p_name = citation_entry['title']
        p_time = citation_entry['year']
        p_author = citation_entry['author']
        if 'journal' in citation_entry.keys():
            p_jrnl = citation_entry['journal']
        elif 'booktitle' in citation_entry.keys():
            p_jrnl = citation_entry['booktitle']
        else:
            p_jrnl = ' '
        browser.back()
        browser.back()

        # 找到第一篇论文作为检索到的论文，打开引用链接
        cited_links = browser.links.find_by_partial_href('/scholar?cites=')
        link = cited_links[0].outer_html.split('="')[1].split('">')[0]
        search_link = GS_root + link[1:]
        browser.visit(search_link)

        while True:
            # 遍历列表，整理引用我们论文的信息
            links = browser.find_by_xpath('//a[@title="引用"]')
            n_links = len(links)
            for ii in range(n_links):
                link = links[ii]
                try:
                    link.click()
                    # 打开bibtex页面 并获取信息
                    browser.find_by_xpath('//a[@class="gs_citi"]').click()
                    bibtex = browser.find_by_xpath('//pre').value
                    citation_entry = bibtexparser.loads(bibtex).entries[0]
                    print(citation_entry)

                    # 整理信息到文件
                    r_name = citation_entry['title']
                    r_time = citation_entry['year']
                    r_author = citation_entry['author']
                    if 'journal' in citation_entry.keys():
                        r_jrnl = citation_entry['journal']
                    elif 'booktitle' in citation_entry.keys():
                        r_jrnl = citation_entry['booktitle']
                    else:
                        r_jrnl = ' '
                    write_csv_line(fid_write, (p_name, p_jrnl, p_time, p_author,
                                            r_name, r_jrnl, r_time, r_author), is_first=is_csv_header)
                    is_csv_header = False

                    browser.back()
                    browser.back()
                    links = browser.find_by_xpath('//a[@title="引用"]')
                except Exception as e:
                    print(e)
                    browser.visit(search_link)
                    links = browser.find_by_xpath('//a[@title="引用"]')
            try:
                browser.find_link_by_text('下一页').click()
            except splinter.exceptions.ElementDoesNotExist as e:
                # 没找到下一页，一篇论文整理结束
                print(f'{paper_name} is finished.')
                break

browser.close()

Processing paper: From intensity profile to surface normal: photometric stereo for unknown light sources and isotropic reflectances
{'publisher': 'Elsevier', 'year': '2015', 'pages': '429--442', 'volume': '320', 'journal': 'Information Sciences', 'author': 'Liu, Anan and Wang, Zhongyang and Nie, Weizhi and Su, Yuting', 'title': 'Graph-based characteristic view set extraction and matching for 3D model retrieval', 'ENTRYTYPE': 'article', 'ID': 'liu2015graph'}
Message: stale element reference: element is not attached to the page document
  (Session info: chrome=80.0.3987.149)

{'publisher': 'IEEE', 'year': '2016', 'pages': '2060--2073', 'number': '10', 'volume': '39', 'journal': 'IEEE Transactions on Pattern Analysis and Machine Intelligence', 'author': 'Hui, Zhuo and Sankaranarayanan, Aswin C', 'title': 'Shape and spatially-varying reflectance estimation from virtual exemplars', 'ENTRYTYPE': 'article', 'ID': 'hui2016shape'}
Message: stale element reference: element is not attached to the

ElementDoesNotExist: no elements could be found with xpath "//a[@title="引用"]"

In [12]:
import time
test_url = 'https://scholar.google.com/scholar?cites=12023925925526822666&amp;as_sdt=2005&amp;sciodt=0,5&amp;hl=zh-CN&google_abuse=GOOGLE_ABUSE_EXEMPTION%3DID%3Dfba1c7ce6762b209:TM%3D1585781820:C%3Dr:IP%3D74.120.174.104-:S%3DAPGng0u6GHSRPKTi-K3OByo1wAv5RbwG8g%3B+path%3D/%3B+domain%3Dgoogle.com%3B+expires%3DThu,+02-Apr-2020+01:57:00+GMT'
browser.visit(test_url)


In [13]:
aa = browser.find_link_by_text('下一页')
print(aa.outer_html)

ElementDoesNotExist: no elements could be found with link by text "下一页"